# Homework 1 - Streaming (10 pts)

### DUE: 06/26/2018 before class at 2:00pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_Streaming_NetID.ipynb.

## Task 1 (5 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [1]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

hiredCabs = 0
maxHiredCabs = 0
for row in csvRows('taxi_events.csv'):
    if row['event']=='pickup':
        hiredCabs += 1
        if maxHiredCabs<hiredCabs:
            maxHiredCabs = hiredCabs
    else:
        hiredCabs -= 1
    
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

<ol>
<li> First, a function is defined to read the CSV file and yield the rows.</li>
<li> Then, global variables are initiated to represent the amount of hired cabs and the maximum number of hired cabs.</li>
<li> A for loop function iterates through the imported dataset, and checks whether or not the event is a pickup event.</li>
<li> If it is not, the function reduces from the counter of hired cabs.</li>
<li> If it is, then the function adds to the counter of hired cabs and assigns the maximum hired cabs variable to that (since it will be always be lees than the current amount of hired cabs). When the iteration is completed, the global variable of maximum hired cabs will show the maximum amount of hired cabs for that iteration.</li>
</ol>

## Task 2 (5 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [84]:
import csv
import pandas as pd
import numpy as np
import datetime

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0

activeTrips = []
activeTrips = pd.Series(activeTrips)

for row in csvRows('taxi_trips.csv'):
    
    rowPick = pd.to_datetime(row['pickup_time'])
    rowDrop = pd.to_datetime(row['dropoff_time'])
    
    activeTrips = activeTrips.append(pd.Series(rowDrop))
    activeTrips = activeTrips[activeTrips > (rowPick)]
    
    activeCabs = len(activeTrips)
    
    if activeCabs > maxHiredCabs:
        maxHiredCabs = activeCabs
   
print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

<ol>
<li>Initiating empty array, converting to Series to keep all the active trips (where the timestamp is greater than the pickup time).</li>
<li>While iterating through the dataset, creating local variables to assignt the timestamp of pickups and dropoffs.</li>
<li>Appending dropoff timestamps to the activeTrips list to compare later.</li>
<li>Iterating through the active trips array keeping only the records where the droppoff time is greater than the pickup time.</li>
<li>At the end of iteration, whatever is the length of the active trips array is the number of maximum hired cabs.</li>
</ol>